In [2]:
import numpy as np
import gym
from collections import deque
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


#### 经验回放池：

In [3]:
class ReplayBuffer:
    def __init__(self, max_batch_size):
        # deque定义的是一个先入先出对列，其效率高于.pop(0)
        self.max_batch_size = max_batch_size
        self.buffer = deque(maxlen=self.max_batch_size)  # 固定容量
        
    def push_transition(self, transition):
        # 这里的实现没有判断是否重复（去重操作适合探索性任务）
        self.buffer.append(transition)
        
    def sample(self, batch_size):
        if batch_size > self.max_batch_size:
            raise ValueError("采样的长度大于经验回放池大小！")
        return random.sample(self.buffer, min(batch_size, len(self.buffer)))
    def len_buffer(self):
        return len(self.buffer)

#### DQN:
1. DQN 算法概述
DQN 是一种深度强化学习算法，结合 Q-Learning 和 深度神经网络，用于处理高维状态空间（如图像或连续状态）。由 DeepMind 在 2013 和 2015 年提出，广泛应用于 Atari 游戏等任务。DQN 使用神经网络近似 Q 值函数 $Q(s, a; \theta)$，通过优化时序差分（TD）误差学习最优策略。

2. 数学原理
2.1 强化学习与马尔可夫决策过程 (MDP)
- **强化学习建模为 MDP $(S, A, P, R, \gamma)$**：
状态 $S$：RollingBall 的 $[x, y, v_x, v_y]$。
动作 $A$：25 个离散动作（Hashposotion.py）。
转移概率 $P(s' | s, a)$：由 GridWorld.py 物理规则隐式定义。
奖励 $R(s, a, s')$：-2.0（每步）、-10.0（撞墙）、+300.0（目标）。
折扣因子 $\gamma$：通常 0.99。

- **目标：最大化累积折扣奖励**：
$$
J(\pi) = \mathbb{E}{\pi} \left[ \sum{t=0}^\infty \gamma^t R(s_t, a_t, s_{t+1}) \right]
$$
平稳分布 $d_\pi(s)$，满足 $d_\pi^T P_\pi = d_\pi^T$，描述策略 $\pi$ 下的长期状态访问概率。

2.2 Q-Learning 与 Bellman 方程
Q 值函数 $Q^\pi(s, a)$ 表示状态 $s$ 下执行动作 $a$，然后按策略 $\pi$ 行动的预期累积奖励：
$$Q^\pi(s, a) = \mathbb{E}{\pi} \left[ R(s, a, s') + \gamma \sum{a'} \pi(a'|s') Q^\pi(s', a') \right]$$
最优 Q 值函数 $Q^*(s, a)$ 满足 Bellman 最优方程（《Chapter 8》, Box 7.5）：
$$Q^(s, a) = \mathbb{E} \left[ R(s, a, s') + \gamma \max_{a'} Q^(s', a') \right]$$
Q-Learning 更新：
$$Q(s, a) \leftarrow Q(s, a) + \alpha \left[ R(s, a, s') + \gamma \max_{a'} Q(s', a') - Q(s, a) \right]$$
RollingBall 的连续状态使 Q 表格不可行，需函数逼近。
2.3 DQN 的函数逼近
DQN 使用神经网络 $Q(s, a; \theta)$ 近似 $Q^*(s, a)$。

表格法：存储所有 $Q(s, a)$，对连续状态不可行。
函数逼近：用参数 $\theta$ 表示 $Q(s, a; \theta)$，存储效率高，泛化能力强。

RollingBall 配置：

输入：4 维状态 $[x, y, v_x, v_y]$。
输出：25 维 Q 值向量。
网络：全连接网络（2-3 层，128 神经元，ReLU 激活）。

DQN 优化 Bellman 误差：
$$J(\theta) = \mathbb{E} \left[ \left( R + \gamma \max_{a'} Q(S', a'; \theta) - Q(S, A; \theta) \right)^2 \right]$$
直接优化不稳定，需以下技术。
3. DQN 的关键技术及数学推导
3.1 经验回放（Experience Replay）
问题：经验 $(s, a, r, s', \text{done})$ 具有时间相关性，违背神经网络训练的 i.i.d. 假设。
解决方案：回放池存储经验，随机采样批量数据。强调均匀采样确保状态-动作对 $(S, A)$ 近似均匀分布。
数学原理：

回放池 $\mathcal{D}$ 存储 $(s, a, r, s', \text{done})$。
采样 $N$ 个经验，损失为：

$$L(\theta) = \frac{1}{N} \sum_{i=1}^N \left[ r_i + \gamma (1 - \text{done}i) \max{a'} Q(s'_i, a'; \theta) - Q(s_i, a_i; \theta) \right]^2$$

3.2 目标网络（Target Network）
问题：目标值 $r + \gamma \max_{a'} Q(s', a'; \theta)$ 随 $\theta$ 变化，训练不稳定。
解决方案：目标网络 $Q(s, a; \theta^-)$，参数 $\theta^-$ 定期复制。《Chapter 8》（Section 8.4.1）描述固定 $\theta^-$ 简化梯度计算：
$$L(\theta) = \mathbb{E} \left[ \left( r + \gamma \max_{a'} Q(s', a'; \theta^-) - Q(s, a; \theta) \right)^2 \right]$$
数学推导：

目标值：

$$y_i = r_i + \gamma (1 - \text{done}i) \max{a'} Q(s'_i, a'; \theta^-)$$

梯度：

$$\nabla_\theta L(\theta) = -\frac{2}{N} \sum_{i=1}^N \left[ y_i - Q(s_i, a_i; \theta) \right] \nabla_\theta Q(s_i, a_i; \theta)$$

每 $k$ 步（如 100），$\theta^- \leftarrow \theta$。


3.3 ε-贪婪策略
问题：需平衡探索与利用。
解决方案：以概率 $\epsilon$ 随机选择动作，否则选择 $\arg\max_a Q(s, a; \theta)$。《Chapter 8》（Box 8.1）提到探索策略（如 ε-贪婪）确保平稳分布唯一。
数学原理：

策略：

$$\pi(a|s) = \begin{cases}\frac{\epsilon}{|A|} + (1 - \epsilon) \cdot \mathbb{I}[a = \arg\max_{a'} Q(s, a'; \theta)] & \text{if } a = \arg\max_{a'} Q(s, a'; \theta) \\frac{\epsilon}{|A|} & \text{otherwise}\end{cases}$$

衰减：

$$\epsilon_t = \epsilon_{\text{end}} + (\epsilon_{\text{start}} - \epsilon_{\text{end}}) \exp(-t / \tau)$$
与 RollingBall 的结合：

env.action_space.sample() 实现随机动作。
衰减 $\epsilon$（如 1.0 到 0.1）鼓励目标探索。

4. DQN 算法流程
综合《Chapter 8》（Algorithm 8.3）和 hrl.boyuai.com：

初始化：

在线网络 $Q(s, a; \theta)$，目标网络 $Q(s, a; \theta^-)$，$\theta^- \leftarrow \theta$。
回放池 $\mathcal{D}$，容量 capacity。
超参数：学习率 $\alpha$，折扣因子 $\gamma$，$\epsilon$ 参数。

交互：

按 ε-贪婪选择 $a$。
执行 $a$，获取 $r, s', \text{done}$。
存储 $(s, a, r, s', \text{done})$。

优化：

采样批量经验。

目标值：
$$y_i = r_i + \gamma (1 - \text{done}i) \max{a'} Q(s'_i, a'; \theta^-)$$

损失：
$$L(\theta) = \frac{1}{N} \sum_{i=1}^N \left[ y_i - Q(s_i, a_i; \theta) \right]^2$$

更新：

每 $k$ 步，$\theta^- \leftarrow \theta$。
衰减 $\epsilon_t$。

5. 局限与改进

Q 值过估计：

$\max_{a'} Q(s', a'; \theta^-)$ 可能高估。《Chapter 10》（Section 10.2）提到基线减少方差，DQN 用 Double DQN：
$$y_i = r_i + \gamma (1 - \text{done}_i) Q(s'i, \arg\max{a'} Q(s'_i, a'; \theta); \theta^-)$$

稀疏奖励：RollingBall 的 +300.0 奖励稀疏，需奖励整形。

离散动作：DQN 限于离散动作

In [4]:
class Q_net(torch.nn.Module):
    """"Q网络使两个MLP进行连接，用于DQN和Double DQN"""
    def __init__(self, input_dim, hidden_dim, output_dim):
        # 调用nn.Module的初始化，确保网络能够正确初始化
        super().__init__()
        self.fc1 = torch.nn.Linear(input_dim, hidden_dim)
        # 一维批归一化（LayerNorm Normalization）层的操作
        self.ln1 = nn.LayerNorm(hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, hidden_dim)
        self.ln2 = nn.LayerNorm(hidden_dim)
        self.fc3 = torch.nn.Linear(hidden_dim, output_dim)
        self._init_weights()
        
    def _init_weights(self):
        """对每一层进行权重初始化"""
        # 隐藏层：使用 ReLU 专用的 Kaiming 初始化
        nn.init.kaiming_uniform_(self.fc1.weight, nonlinearity='relu')
        # 初始化全连接层偏置，稳定初始输出，加速早期训练。
        nn.init.zeros_(self.fc1.bias)
        nn.init.kaiming_uniform_(self.fc2.weight, nonlinearity='relu')
        nn.init.zeros_(self.fc2.bias)
        # 输出层受限初始化
        bound = 3 / np.sqrt(self.fc3.weight.size(0))
        nn.init.uniform_(self.fc3.weight, -bound, bound)
        nn.init.zeros_(self.fc3.bias)
        
    def forward(self, x):
        """定义从输入状态到输出 Q 值的计算流程。
            输入 x：状态张量，形状为 (batch_size, input_dim)
        """
        x = self.ln1(F.relu(self.fc1(x)))
        x = self.ln2(F.relu(self.fc2(x)))
        return self.fc3(x)

1. self.bn1 = nn.BatchNorm1d(hidden_dim) 是定义一个一维批归一化（Batch Normalization）层的操作。
- **核心功能**针对全连接层（Linear）或一维卷积层（Conv1D）的输出.对每个隐藏层神经元的输出进行标准化处理：
    $$
    \hat{x}_i = \frac{x_i - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}} \quad \text{(标准化)}
    $$
    $$
    y_i = \gamma \hat{x}_i + \beta \quad \text{(可学习缩放和平移)}
    $$
  其中：
$μ_B$是该神经元在当前batch上的均值,$σ_B^2$是方差,γ(weight)和β(bias)是可学习的参数。

- **在DQN中的具体作用:**

    正向传播时：标准化过程计算当前batch中每个神经元的均值和方差，对128个神经元的输出独立标准化，通过γ和β保留网络的表达能力

    反向传播时：自动更新当前batch的$μ_B$和$σ_B^2$、可学习参数γ和β、全局统计量（用于推理的running_mean和running_var）

- **数据流维度变化：**

    输入: [batch_size, 4] 
    
    fc1: [batch_size, 128] 

    bn1: [batch_size, 128] (标准化后)

    fc2: [batch_size, 128]

    输出: [batch_size, 25]
    通过这种设计，BatchNorm1d能有效提升DQN在RollingBall环境中的训练稳定性和收敛速度。

---

In [ ]:
class DQN(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim, action_dim, action_range, lr, gamma, 
                 epsilon, device, seed=None):
        super().__init__()
        # 初始化参数
        self.action_dim = action_dim
        self.state_dim = state_dim
        self.hidden_dim = hidden_dim
        self.action_range = action_range # 动作范围
        self.epsilon = epsilon # spsilon-greedy探索率
        self.gamma = gamma # 折扣率
        self.lr = lr
        self.rng = np.random.RandomState(seed) 
        self.device = device
        # 初始化训练网络
        self.Qnet = Q_net(state_dim, hidden_dim, action_dim).to(device)
        # 初始化目标网络
        self.Qnet_target = Q_net(state_dim, hidden_dim, action_dim).to(device)
        # 初始化使用Adam更新器(仅优化在线训练网络)
        self.optimizer = torch.optim.Adam(self.Qnet.parameters(), lr=lr)
    
    def max_q_value_of_state(self, state):
        """计算给定状态下所有动作的最大的Q值，评估策略质量"""
        state = torch.tensor(state, dtype=torch.float32).to(self.device)
        return self.Qnet(state).max().item()

    def take_action(self, state):
        """采取动作,使用ε-greedy策略"""
        if self.rng.random() < self.epsilon:
            action = self.rng.randint(self.action_range)
        else:
            state = torch.tensor(state, dtype=torch.float,).to(self.device)
            action = self.Qnet(state).argmax().item()
        return action
    
    def update(self, transition_dict, tau=0.01):
        # 将经验转化成张量：states,next_states-[batch_size, state_dim],actions-[batch_size, 1],rewards,done-[(batch_size)]
        states = torch.tensor(transition_dict['states'], dtype=torch.float32).to(self.device)
        actions = torch.tensor(transition_dict['actions'], dtype=torch.int64).view(-1, 1).to(self.device)
        # 创建张量->升维，[batch_size, 1]（该形式符合自动广播机制）->设备转移->降维tensor([1,2,3])
        rewards = torch.tensor(transition_dict['rewards'], dtype=torch.float32).view(-1,1).to(self.device).squeeze()
        next_states = torch.tensor(transition_dict['next_states'], dtype=torch.float).to(self.device)
        dones = torch.tensor(transition_dict['dones'], dtype=torch.float32).view(-1,1).to(self.device).squeeze()
        
        # Q值计算and目标Q值计算
        # 计算当前状态-动作对的 Q 值 Q(s_i, a_i; θ)。
        q_values = self.Qnet(states).gather(dim=1, index = actions).squeeze()
        max_next_q_valus = self.Qnet_target(next_states).max(axis=1)[0] # 取每行最大值
        # 结合未来奖励与未来Q值，并考虑是否终止
        q_targets = rewards + self.gamma * max_next_q_valus * (1 - dones)
        
        # 计算训练损失(q_values，q_targets均为（batch_size,actions_dim）)
        DQN_Loss = F.mse_loss(q_values, q_targets, reduction='mean') # 计算 Q 值与目标的 MSE.（默认自动平均）
        self.optimizer.zero_grad() # 清空梯度,避免梯度累加（PyTorch默认会累加梯度，训练时必须手动清零）
        DQN_Loss.backward() # 自动计算损失对网络参数的梯度,梯度存储在参数的 .grad 属性中
        self.optimizer.step() # 根据梯度更新网络
        
        # 目标网络更新(采用软更新)(网络中使用layernorm，如果使用batchnorm需要将running.mean和runnig.var进行软更新)
        for target_param, param in zip(self.Qnet_target.parameters(), self.Qnet.parameters()):
            target_param.data.copy_(tau * param + (1-tau)*target_param)
        """
        若网络同时包含batchnorm和layernorm:使用一下方案
        for target_param, param in zip(self.Qnet_target.modules(), self.Qnet.modules()):
            # 更新普通参数
            if hasattr(target_param, 'weight') and not isinstance(target_param, (nn.BatchNorm1d, nn.LayerNorm)):
                target_param.weight.data.copy_(tau*param.weight + (1-tau)*target_param.weight)
            # 更新BatchNorm的running统计量
            if isinstance(tgtarget_paramt, nn.BatchNorm1d):
                target_param.running_mean.copy_(tau*param.running_mean + (1-tau)*target_param.running_mean)
                target_param.running_var.copy_(tau*param.running_var + (1-tau)*target_param.running_var)
        """
        